In [1]:
from pyspark.sql.session import SparkSession
import pandas as pd
import pyspark.sql.functions as f
from pyspark.dbutils import DBUtils
import plotly.express as px
from pyspark.sql.window import Window
%matplotlib inline


In [2]:
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

In [3]:
def display(df):
    import pandas as pd
    pd.options.display.max_columns = None
    pd.options.display.max_rows = False
    return(df.limit(1000).toPandas())

In [6]:
spark.table('alocacao_automatica.tb_cad_gm')

DataFrame[GM: int, GM_DESC: string, DCO: int, DCO_DESC: string, GG: string, SALA: string, MTA_MTO: string, FAMILIA_PROD: string, AGRUP_LIVRE: string, INVERNO: string, BASICO_COLECAO: string, MODELO: string, TIPO_PROD: string, GERENTE_ESTILO: string, MARCA: string, GERENTE_CATEG: string, GERENTE_PROD: string, PROGRAMA: string, AGRUP_LIVRE2: string, GG2: string, GG3: string, GG4: string, EXCL_SITE: string, TP_INVERNO: string, MUNDO: string, DCO_AGRUPADOR: string, GG_OLD: string, SEGMENTO: string, JEANS: string]

In [7]:
df_tb_cad_gm = spark.table('alocacao_automatica.tb_cad_gm')

In [8]:
df_tb_cad_gm = df_tb_cad_gm.groupBy('GG').count().orderBy(f.col('count').desc())

In [9]:
df_tb_cad_gm = df_tb_cad_gm.withColumn('Média',f.avg('count').over(Window.partitionBy()))
display(df_tb_cad_gm)

,GG,count,Média
0,INFANTIL,476,211.5
1,CASA RIACHUELO,390,211.5
2,FEMININO,323,211.5
3,MASCULINO,269,211.5
4,LP,205,211.5
5,CBA,204,211.5
6,TECNOLOGIA & BELEZA,79,211.5
7,BODY WORK,71,211.5
8,CARTERS,57,211.5
9,FANLAB,41,211.5


In [10]:
df = df_tb_cad_gm.toPandas()
# Create the line chart
fig = px.bar(df, x='GG', y='count',text='count')
fig.add_scatter(x=df['GG'], y=df['Média'], mode='lines', name='Média', line=dict(color='red'))
# Show the chart
fig.show()

In [ ]:
!databricks fs cp C:/Users/Eduardo/Desktop/dim_gender.csv dbfs:/mnt/teste/dim_gender.csv

In [10]:
dbutils.fs.ls('./mnt/teste')

[FileInfo(path='dbfs:/mnt/teste/dim_category.csv', name='dim_category.csv', size=467),
 FileInfo(path='dbfs:/mnt/teste/dim_gender.csv', name='dim_gender.csv', size=34)]

In [11]:
spark.read.csv('dbfs:/mnt/teste/dim_gender.csv',header=True).show()

+---------+------+
|gender_id|gender|
+---------+------+
|        1|  male|
|        2|female|
+---------+------+



In [11]:
df = spark.read.option('header','true').format('csv').load("dbfs:/mnt/teste/dim_category.csv")
display(df) #

,category_id,category_L1,category_L2
0,1,Furniture,Bookcases
1,2,Furniture,Chairs
2,3,Furniture,Tables
3,4,Furniture,Furnishings
4,5,Office Supplies,Labels
5,6,Office Supplies,Storage
6,7,Office Supplies,Art
7,8,Office Supplies,Binders
8,9,Office Supplies,Appliances
9,10,Office Supplies,Paper
